In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
stop_words = set(stopwords.words('english')) 
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Input
from keras.models import Model
from keras import optimizers
from keras.layers import concatenate,CuDNNLSTM
from keras import initializers
from keras import backend as K
import random

from keras.layers import Dense, LSTM,Dropout,MaxPooling1D, Activation,Bidirectional,Reshape,Permute,Conv2D,Multiply,Flatten,Lambda,Layer,TimeDistributed,CuDNNLSTM

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Using TensorFlow backend.


In [0]:
'''
Breaking whole data into list of sentences
'''
def seperate_sentences(data):
    return sent_tokenize(data) 


def Sentence_to_POS(all_sentences):

    data=[]
    for sentence in all_sentences: 
        one_sentence_pos = []
        '''
        Word tokenizers is used to find the words and punctuation in a string 
        '''  
        wordsList = nltk.word_tokenize(sentence) 

        '''
        Removing stop words from wordList.
        ''' 
        wordsList = [w for w in wordsList if not w in stop_words]  

        '''
        Using a Tagger. Which is part-of-speech tagger or POS-tagger.  
        '''

        tagged = nltk.pos_tag(wordsList) 
        for x in tagged:
            one_sentence_pos.append(x[1])
        data.append(one_sentence_pos)

    return data


def Fianl_X_and_Y(novel_POS,novel_label,number_of_sentenes_in_one):
    '''
    This funtion takes generates more data for training by breaking one novel into multiple novels
    with k number of sentences (number_of_sentenes_in_one) .we are ignoring some last sentences if they
    dont fit
    '''
    i=0
    X=[]
    Y=[]
    for novel in novel_POS:
        start = 0
        end = number_of_sentenes_in_one
        while(end<=len(novel)):
            X.append(novel[start:end])
            Y.append(novel_label[i])
            start=end
            end=end+number_of_sentenes_in_one
        i+=1
    return X,Y


def Get_X_and_Y_In_POS_Form(path_of_dataset_directory,number_of_sentences_in_one):
    '''
    this function takes the path of directory where the dataset is and returns the processed X and Y.
    The function calls will let u understand more the flow of code
    '''
#     all_files = os.listdir(path_of_dataset_directory)
    all_files = ['12ItrainA1.TXT','12ItrainA2.TXT','12ItrainB1.TXT','12ItrainB2.TXT','12ItrainC1.TXT','12ItrainC2.TXT','12ItrainD1.TXT','12ItrainD2.TXT'
                ,'12ItrainE1.TXT','12ItrainE2.TXT','12ItrainF1.TXT','12ItrainF2.TXT','12ItrainG1.TXT','12ItrainG2.TXT','12ItrainH1.TXT','12ItrainH2.TXT'
                ,'12ItrainI1.TXT','12ItrainI2.TXT','12ItrainJ1.TXT','12ItrainJ2.TXT','12ItrainK1.TXT','12ItrainK2.TXT','12ItrainL1.TXT','12ItrainL2.TXT'
                 ,'12ItrainM1.TXT','12ItrainM2.TXT','12ItrainN1.TXT','12ItrainN2.TXT']
    novel_POS=[]
    novel_label=[]
    file_number=1
    for filename in all_files:
        print(file_number,"/",len(all_files)," Done")
        file_number+=1
        full_path = '/content/gdrive/My Drive/dataset_smai/'+filename
        with open(full_path, 'r') as f:
            data = f.read().replace('"\n"','').replace('\n',' ').replace('- ','')
            all_sentences = seperate_sentences(data)
            sentences_to_pos = Sentence_to_POS(all_sentences)
            novel_POS.append(sentences_to_pos)
            novel_label.append(filename[8])
    return Fianl_X_and_Y(novel_POS,novel_label,number_of_sentences_in_one)

def tag_to_index_dictionary(X):
    '''
    This function just creates a dictionary from the POS tags which we
    encountered in our dataset.As the network works with numbers so simple will make
    dictionary which stores the index of associated POS tag
    '''
    tag = set([])
    for doc in X:
        for sentence in doc:
            for word in sentence:
                tag.add(word)
    tag2index = {t: i + 1 for i, t in enumerate(list(tag))}
    tag2index['-PAD-'] = 0
    return tag2index

def convert_tag_to_sequence_numbers(X):
    tag2index = tag_to_index_dictionary(X)
    '''
    using the tag2index dictionary assign indexs to the POS tags in our data
    '''
    new_X=[]
    for doc in X:
        new_S = []
        for sentence in doc:
            new_W=[]
            for word in sentence:
                new_W.append(tag2index[word])
            new_S.append(new_W)
        new_X.append(new_S)
    return new_X


def pad_zeros_to_sequence(X,max_length):
    '''
    This function padds zeros to the sequences so as to make fixed sequences
    According to paper 15 is best value for max_length
    '''
    
    new_X=[]
    for doc in X:
        new_X.append(pad_sequences(doc, maxlen=max_length, padding='post'))
    return new_X

def Encode_Labels(Y):
    '''
    This fucntion encodes the labels i.e., assign numbers to the labels
    '''
    le = LabelEncoder()
    return le.fit_transform(Y)




In [5]:
print("Preprocessing Phase 1 : Fetching data")
print(".....................................")

X__,Y__ = Get_X_and_Y_In_POS_Form('dataset/three_author_dataset',100)

print("Preprocessing Phase 1 : Finished\n\n")
print(".....................................")

Preprocessing Phase 1 : Fetching data
.....................................
1 / 28  Done
2 / 28  Done
3 / 28  Done
4 / 28  Done
5 / 28  Done
6 / 28  Done
7 / 28  Done
8 / 28  Done
9 / 28  Done
10 / 28  Done
11 / 28  Done
12 / 28  Done
13 / 28  Done
14 / 28  Done
15 / 28  Done
16 / 28  Done
17 / 28  Done
18 / 28  Done
19 / 28  Done
20 / 28  Done
21 / 28  Done
22 / 28  Done
23 / 28  Done
24 / 28  Done
25 / 28  Done
26 / 28  Done
27 / 28  Done
28 / 28  Done
Preprocessing Phase 1 : Finished


.....................................


In [0]:
X,Y=X__,Y__

In [7]:
np.unique(Y,return_counts=True)

(array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
        'N'], dtype='<U1'),
 array([ 95, 264, 191, 125, 180,  72, 100, 159,  89, 104,  79,  47,  60,
         74]))

In [0]:
def Balance_Data(X,Y):
  MIN_VAL = min(np.unique(Y,return_counts=True)[1])+200
  
  attri_seen = {}
  attri_seen['A']=attri_seen['B']=attri_seen['C']=attri_seen['D']=attri_seen['E']=attri_seen['F']=attri_seen['G']=attri_seen['H']=attri_seen['I']=attri_seen['J']=attri_seen['K']=attri_seen['L']=attri_seen['M']=attri_seen['N']=0
  X_data=[]
  for i in range(len(X)):
      if(attri_seen[Y[i]]<MIN_VAL):
          X_data.append([X[i],Y[i]])
          attri_seen[Y[i]]+=1
    
  random.shuffle(X_data)
        
  new_X=[]
  new_Y=[]
  for m , n in X_data:
    new_X.append(m)
    new_Y.append(n)
    
  return new_X,new_Y

X,Y=Balance_Data(X,Y)

In [0]:
DICTIONARY_LENGTH = len(tag_to_index_dictionary(X))
MAX_LENGTH = 30
SENTENCES_IN_NOVEL=100

X = convert_tag_to_sequence_numbers(X)
X = pad_zeros_to_sequence(X,MAX_LENGTH)
X = np.array(X)
TOTAL_ROWS=X.shape[0]

In [0]:
real_X = []
for i in range(SENTENCES_IN_NOVEL):
    real_X.append(list())
for i in range(SENTENCES_IN_NOVEL):
    for j in range(TOTAL_ROWS):
        real_X[i].append(X[j][i])

X_train = []
for i in range(SENTENCES_IN_NOVEL):
    X_train.append(np.array(real_X[i]))



Y = Encode_Labels(Y) 
from tensorflow.keras.utils import to_categorical
y_binary = to_categorical(Y)


In [0]:
from keras.engine.topology import Layer as LL
class Attention(LL):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        print(self.W_constraint)
        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))
        print(eij.shape,self.b.shape)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [12]:
from keras import  regularizers,constraints,initializers
model = Sequential() 

FIRST_LSTM =256
SECOND_LSTM=256
EMBEDD_VECTOR_LENGTH=128
outputs=[]
inputs_=[]
EMB = Embedding(DICTIONARY_LENGTH, EMBEDD_VECTOR_LENGTH, input_length=30)
cnn_layers=[]
for i in range(29):
  cnn_layers.append(Conv2D(1,[i+1,128],activation='relu',use_bias=True,input_shape=(30,128,1)))

BDR = Bidirectional(CuDNNLSTM(FIRST_LSTM,return_sequences=True),merge_mode='concat')
TDB = TimeDistributed(Dense(DICTIONARY_LENGTH+3))    
for i in range(SENTENCES_IN_NOVEL):# SENTENCES_IN_NOVEL = 100
    if(i%10==0):
      print(i,"/",SENTENCES_IN_NOVEL)
    inputlayer = Input(shape=[MAX_LENGTH]) #MAX_LENGTH = 30 (30 words per sentence)
    inputs_.append(inputlayer)
    layer = EMB(inputlayer)
    
    layer = Reshape((30,128,1),input_shape=(30,128))(layer)
    multiple_cnn=[]
    for i in range(29):
      cnn_2_ = cnn_layers[i](layer)
      cnn_2_ = Reshape((30-i,1),input_shape=(30-i,1,1))(cnn_2_)
#       print(cnn_2_.shape)
      cnn_2_ = MaxPooling1D(pool_size=30-i)(cnn_2_)
#       print(cnn_2_.shape)
      multiple_cnn.append(cnn_2_)
       
    layer = concatenate(multiple_cnn)
            

    outputs.append(layer)
    
merge_ = concatenate(outputs)
print(merge_.shape)
merge_ = Reshape((SENTENCES_IN_NOVEL, 29), input_shape=(SENTENCES_IN_NOVEL*29,))(merge_)
merge_ = Bidirectional(CuDNNLSTM(SECOND_LSTM,return_sequences=True),merge_mode='concat')(merge_)
# merge_ = TimeDistributed(Dense(128))(merge_)

attention_mul = Attention(100)(merge_)#step_dimen =100
output = Dense(14, activation='softmax')(attention_mul)

model = Model(inputs=inputs_,outputs=output)
model.summary()

0 / 100
Instructions for updating:
Colocations handled automatically by placer.
10 / 100
20 / 100
30 / 100
40 / 100
50 / 100
60 / 100
70 / 100
80 / 100
90 / 100
(?, 1, 2900)
None
(?, 100) (100,)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
input_4 (Inpu

In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Nadam(lr=0.001),
              metrics=['accuracy'])
history = model.fit(X_train,y_binary, epochs=40,validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Train on 1459 samples, validate on 163 samples
Epoch 1/40
Train on 1459 samples, validate on 163 samples
Epoch 1/40
1459/1459 [==============================] - 161s 110ms/step - loss: 2.4801 - acc: 0.1995 - val_loss: 2.1241 - val_acc: 0.2577
Epoch 2/40
Epoch 2/40
1459/1459 [==============================] - 36s 25ms/step - loss: 1.9118 - acc: 0.3139 - val_loss: 2.4971 - val_acc: 0.2331
Epoch 3/40
Epoch 3/40
1459/1459 [==============================] - 36s 25ms/step - loss: 1.7423 - acc: 0.3681 - val_loss: 1.6037 - val_acc: 0.4110
Epoch 4/40
Epoch 4/40
1459/1459 [==============================] - 36s 25ms/step - loss: 1.6079 - acc: 0.3982 - val_loss: 1.5111 - val_acc: 0.3926
Epoch 5/40
Epoch 5/40
1459/1459 [==============================] - 36s 25ms/step - loss: 1.5711 - acc: 0.4099 - val_loss: 1.6352 - val_acc: 0.3926
Epoch 6/40
Epoch 6/40
1459/1459 [==============================] - 36s 25